In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import math
import matplotlib.pyplot as plt

In [22]:
d1 = pd.read_excel("Data/World Happiness Report/WHR2018.xlsx", sheet_name='Table2.1')
d2 = pd.read_excel("Data/World Happiness Report/WHR2018.xlsx", sheet_name='Figure2.2')
d3 = pd.read_excel("Data/World Happiness Report/WHR2018.xlsx", sheet_name='Figure2.3')
d4 = pd.read_excel("Data/World Happiness Report/WHR2018.xlsx", sheet_name='Figure2.4')
d5 = pd.read_excel("Data/World Happiness Report/WHR2018.xlsx", sheet_name='SupportingFactors')

In [23]:
print(d1.shape)
d1 = d1.rename(columns={'country':'Country'})
# d1['Average'] = d1.groupby('Country')['Life Ladder'].transform('mean')
# d1['Rank'] = d1['Average'].rank(method='dense', ascending=False)
d1.head()

(1562, 19)


,Country,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Confidence in national government,Democratic Quality,Delivery Quality,Standard deviation of ladder by country-year,Standard deviation/Mean of ladder by country-year,GINI index (World Bank estimate),"GINI index (World Bank estimate), average 2000-15","gini of household income reported in Gallup, by wp5-year"
0,Afghanistan,2008,3.723590,7.168690,0.450662,49.209663,0.718114,0.181819,0.881686,0.517637,0.258195,0.612072,-1.929690,-1.655084,1.774662,0.476600,NaN,NaN,NaN
1,Afghanistan,2009,4.401778,7.333790,0.552308,49.624432,0.678896,0.203614,0.850035,0.583926,0.237092,0.611545,-2.044093,-1.635025,1.722688,0.391362,NaN,NaN,0.441906
2,Afghanistan,2010,4.758381,7.386629,0.539075,50.008961,0.600127,0.137630,0.706766,0.618265,0.275324,0.299357,-1.991810,-1.617176,1.878622,0.394803,NaN,NaN,0.327318
3,Afghanistan,2011,3.831719,7.415019,0.521104,50.367298,0.495901,0.175329,0.731109,0.611387,0.267175,0.307386,-1.919018,-1.616221,1.785360,0.465942,NaN,NaN,0.336764
4,Afghanistan,2012,3.782938,7.517126,0.520637,50.709263,0.530935,0.247159,0.775620,0.710385,0.267919,0.435440,-1.842996,-1.404078,1.798283,0.475367,NaN,NaN,0.344540


In [24]:
print(d2.shape)
d2 = d2.iloc[0:157, 0:11].sort_values(by=['Country']).reset_index(drop=True)
d2.head()

(157, 24)


,Country,Happiness score,Whisker-high,Whisker-low,Dystopia (1.92) + residual,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption
0,Afghanistan,3.6315,3.714525,3.548475,2.195918,0.331814,0.537191,0.254772,0.084552,0.191004,0.036268
1,Albania,4.5860,4.695393,4.476607,1.462148,0.915782,0.817405,0.790387,0.418669,0.149213,0.032436
2,Algeria,5.2946,5.406491,5.182709,2.208535,0.978857,1.153656,0.686599,0.077280,0.055195,0.134517
3,Angola,3.7948,3.951442,3.638158,1.530531,0.730157,1.125020,0.268986,0.000000,0.078868,0.061276
4,Argentina,6.3880,6.485014,6.290986,2.418201,1.072671,1.468341,0.743585,0.569710,0.061942,0.053507


In [25]:
print(d3.shape)
d3= d3.sort_values(by=['Country']).reset_index(drop=True)
d3.head()

(141, 4)


,Country,Changes in happiness scores,Whisker-high,Whisker-low
0,Afghanistan,-0.688338,-0.579448,-0.797228
1,Albania,-0.791441,-0.640828,-0.942054
2,Algeria,-0.168929,-0.008211,-0.329647
3,Argentina,0.112430,0.247466,-0.022607
4,Armenia,-0.078384,0.046207,-0.202976


In [26]:
print(d4.shape)
d4= d4.sort_values(by=['Country']).reset_index(drop=True)
d4.head()

(117, 5)


,Country,Average happiness of foreign born,Whisker-high,Whisker-low,Average happiness of locally born
0,Afghanistan,4.068487,4.268106,3.868869,3.853136
1,Argentina,5.843226,6.077020,5.609432,6.439717
2,Armenia,4.100641,4.261732,3.939550,4.417696
3,Australia,7.249348,7.331693,7.167004,7.323904
4,Austria,6.902624,7.076249,6.728999,7.260852


In [27]:
print(d5.shape)
d5 = d5.sort_values(by=['country']).reset_index(drop=True)
d5 = d5.rename(columns={'country':'Country'})
d5.head()

(156, 15)


,Country,Region indicator,"Life ladder, 2015-2017","Standard error, life ladder, 2015-2017","Log of GDP per person, 2015-2017","GDP per person, 2015-2017","Healthy life expectancy, 2015-2017","Social support, 2015-2017","Standard error, social support, 2015-2017","Freedom to make life choices, 2015-2017","Standard error, freedom to make life choices, 2015-2017","Generosity, 2015-2017, without adjustment for GDP per person","Standard error, generosity, 2015-2017","Perceptions of corruption, 2015-2017","Standard error, perceptions of corruption, 2015-2017"
0,Afghanistan,South Asia,3.631519,0.042360,7.462610,1741.687500,52.013329,0.525075,0.011119,0.445294,0.011449,0.179054,0.008331,0.879705,0.006009
1,Albania,Central and Eastern Europe,4.586040,0.055813,9.338126,11363.095703,68.871552,0.639576,0.010078,0.726340,0.009421,0.259975,0.009048,0.886778,0.005775
2,Algeria,Middle East and North Africa,5.294638,0.057087,9.540703,13914.723633,65.604858,0.776977,0.011171,0.439177,0.019161,0.128988,0.012327,0.698343,0.019293
3,Angola,Sub-Saharan Africa,3.794838,0.079919,8.741957,6260.132812,52.460709,0.765275,0.015744,0.374173,0.018346,0.106829,0.011499,0.833540,0.016014
4,Argentina,Latin America and Caribbean,6.387958,0.049497,9.842001,18807.310547,67.398483,0.905565,0.006363,0.853390,0.007526,0.163174,0.007777,0.847882,0.006881
